# Задача 1, вариант 4
$X_1, \dots, X_n \sim N(0, 2)$, $Y_1, \dots, Y_n \sim N(0, 1)$

$\hat{\tau} = \frac{\hat\sigma_x^2}{\hat\sigma_y^2} = \frac{m\sum_i (X_i - \mu_x)^2}{n\sum_i (Y_i - \mu_y)^2}$ ($\tau$ = 2)

Чтобы сгенерировать $\hat{\tau}$ мы будем генерировать выборку из $X$ и $Y$ размером 25 (потом 10000), считать отношение эмпирических дисперсий. Такую генерацию повторим 1000 раз и посчитаем, сколько раз истинное $\tau$ попадет в заранее построенный доверительный интервал.

Заметим, что $\frac{m\sum X_i^2}{n\sum Y_i^2} \cdot \frac{\sigma_y^2}{\sigma_x^2} = \frac{\hat{\tau}}{\tau}$, где $\frac{\hat{\tau}}{\tau}$ распределена как Fisher(n, m)

$P(F_1 < \frac{\hat{\tau}}{\tau} < F_2) = P(\frac{\hat{\tau}}{F_2} < \tau <\frac{\hat{\tau}}{F_1})$, где $F_1$ и $F_2$ - квантили 97.5% и 2.5% для распределения Фишера: $F_\text{Fisher}^{-1}(0.025)$ и $F_\text{Fisher}^{-1}(0.975)$

In [1]:
import random
import scipy.stats as stats

sigma2_x = 2
sigma2_y = 1
tau_real = sigma2_x / sigma2_y

def gen_X():
    return random.gauss(0, sigma2_x ** 0.5)

def gen_Y():
    return random.gauss(0, sigma2_y ** 0.5)

def gen_tau(n, m):
    sum_x_sq = sum(gen_X() ** 2 for _ in range(n))
    sum_y_sq = sum(gen_Y() ** 2 for _ in range(m))
    return (m * sum_x_sq) / (n * sum_y_sq)

def simulation(n, m, num_experiments=1000, confidence_level=0.95):
    alpha = 1 - confidence_level
    F_low = stats.f.ppf(alpha / 2, n, m) # F^(-1)
    F_high = stats.f.ppf(1 - alpha / 2, n, m)
    
    cover_count = 0
    
    for _ in range(num_experiments):
        F_value = gen_tau(n, m)
        ci_low = F_value / F_high
        ci_high = F_value / F_low
        
        if ci_low <= tau_real <= ci_high:
            cover_count += 1
    
    coverage_rate = cover_count / num_experiments
    return coverage_rate

def experiment(index, n, m, num_experiments=1000, confidence_level=0.95):
    print(f"{index}) n = {n}, m = {m}")
    coverage = simulation(n, m, num_experiments=1000, confidence_level=0.95)
    print(f"Coverage: {coverage:.4f} ({abs(coverage - 0.95):.4f} from 0.95)")
    print()

experiment(1, 25, 25)
experiment(2, 10000, 10000)
experiment(3, 10, 10000)

1) n = 25, m = 25
Coverage: 0.9450 (0.0050 from 0.95)

2) n = 10000, m = 10000
Coverage: 0.9530 (0.0030 from 0.95)

3) n = 10, m = 10000
Coverage: 0.9450 (0.0050 from 0.95)



Сделав несколько повторений всего эксперимента, можно заметить, что почти всегда получается значение около 0.95, вне зависимости от размера выборки для $X$ и $Y$.

Это понятно: точность доверительного интервала зависит от размера выборки для $\tau$, но не от размера выборки для $X$ и $Y$ - эти размеры учитывает распределение Фишера в виде параметров.

Также, что несложно увидеть, доверительный интервал покрывает реальное значение с той же частотой, на основе которой он построен (в данном случае 0.95).